### Longitudinal matching in the CPS

Brian Dew, @bd_econ<br>
Updated: April 12, 2018

-----

Small conceptual example: match records across two months first by ID and then by sex, race, and age. Select the subset of people who were not in the labor force due to disability or illness in month one but are in another category in month two.


##### Resources:
http://www.nber.org/papers/t0247.pdf<br>
http://trim3.urban.org/documentation/input/concepts%20and%20procedures/CPSMatchingSurveys.php


In [38]:
import pandas as pd
import numpy as np

In [52]:
# Read 2018 CPS data into pandas dataframe
df = pd.read_feather('data/cps_2018.ft', nthreads=3)

# Keep only two months
df = df[df['HRMONTH'].isin([1, 2])]

In [53]:
# Create a unique ID based
df['UniqueID'] = (df['HRHHID'].astype(str) + 
                  df['HRHHID2'].astype(str) + 
                  df['PULINENO'].astype(str))

# Identify disabled or ill as variable
df['disill'] = np.where(((df['PRWNTJOB']==2) & 
                        ((df['PEMLR']==6) | 
                         (df['PENLFACT'].isin([1, 2])))), 1, 0)

In [54]:
# Groups based on month and month in survey
g1mo1 = df[(df['HRMIS'].isin([1, 5])) & (df['HRMONTH'] == 1)]
g2mo1 = df[(df['HRMIS'].isin([2, 6])) & (df['HRMONTH'] == 1)]
g2mo2 = df[(df['HRMIS'].isin([2, 6])) & (df['HRMONTH'] == 2)]
g3mo1 = df[(df['HRMIS'].isin([3, 7])) & (df['HRMONTH'] == 1)]
g3mo2 = df[(df['HRMIS'].isin([3, 7])) & (df['HRMONTH'] == 2)]
g4mo2 = df[(df['HRMIS'].isin([4, 8])) & (df['HRMONTH'] == 2)]

In [55]:
# So-called naive match based on id and month in sample
naive = (pd.merge(g1mo1, g2mo2, on='UniqueID', how='inner').append(
         pd.merge(g2mo1, g3mo2, on='UniqueID', how='inner')).append(
         pd.merge(g2mo1, g3mo2, on='UniqueID', how='inner')).append(
         pd.merge(g3mo1, g4mo2, on='UniqueID', how='inner')))

In [56]:
# Keeping the observations with the same sex and race
# Age should be no more than one year more than prev. mo.
data = naive[(naive['PESEX_x'] == naive['PESEX_y']) &
             (naive['PRDTRACE_x'] == naive['PRDTRACE_y']) &
             (naive['PRTAGE_x'] <= naive['PRTAGE_y'] + 1) &
             (naive['PRTAGE_y'] >= naive['PRTAGE_x'])]

In [57]:
# This part will require careful thought, but here's a start
data[data['disill_x'] > data['disill_y']]

,HRHHID_x,HRMONTH_x,HRYEAR4_x,HRMIS_x,HRHHID2_x,GESTFIPS_x,GTMETSTA_x,PRTAGE_x,PESEX_x,PEEDUCA_x,...,PRERNHLY_y,PRERNWA_y,PENLFRET_y,PENLFACT_y,PWORWGT_y,PWSSWGT_y,PRCHLD_y,PWCMPWGT_y,WBHAO_y,disill_y
22,31817006100046,1,2018,5,6011,1,1,59,1,43,...,-1,-1,-1,-1,0,20375582,0,21020585,1,0
314,59071076004611,1,2018,5,6111,1,2,42,2,40,...,-1,-1,-1,-1,0,12630492,3,12836390,1,0
569,309015962210563,1,2018,1,8011,2,1,52,2,44,...,-1,-1,-1,-1,0,5245895,0,5353668,1,0
1085,840866110903607,1,2018,5,6011,4,1,61,1,39,...,-1,-1,-1,-1,0,28213557,0,28456081,1,0
1361,28025469002021,1,2018,1,8011,5,2,33,1,41,...,-1,-1,-1,-1,0,11587374,0,11290149,1,0
1383,148690050029820,1,2018,1,8011,5,2,32,1,43,...,-1,-1,-1,4,0,11587374,3,11508348,1,0
1521,621751002200393,1,2018,1,8111,5,1,56,2,43,...,-1,-1,-1,-1,0,13662704,0,13846572,1,0
1546,610663300698,1,2018,1,8011,6,1,52,1,41,...,-1,-1,2,4,0,37348022,0,37257370,1,0
1705,27390419014675,1,2018,5,6011,6,1,51,1,40,...,-1,-1,2,6,0,35109721,0,35024501,1,0
1753,61365009700984,1,2018,1,8011,6,1,76,2,42,...,-1,-1,-1,-1,0,31106647,0,31429501,1,0
